
# Web Scraping Ticket Prices from TicketWeb

This Jupyter Notebook is designed to extract ticket sales data from an Excel file, search for ticket information on TicketWeb, and analyze the data. The workflow involves the following steps:

1. **Setup and Initialization**:
    - Import necessary libraries such as `selenium`, `pandas`, and `numpy`.
    - Configure Selenium WebDriver with specific options for web scraping.

2. **Data Loading**:
    - Load ticket sales data from an Excel file into a pandas DataFrame.

3. **Web Scraping**:
    - Define a function to generate TicketWeb search URLs based on artist names.
    - Iterate through the DataFrame to search for ticket information on TicketWeb.
    - Extract ticket tier and price information for upcoming events.

4. **Data Aggregation**:
    - Combine the extracted ticket information into a new DataFrame.
    - Save the aggregated data back to the Excel file.

5. **Data Display**:
    - Display the final DataFrame containing the ticket information.

This project uses Selenium WebDriver for web scraping.


In [1]:

from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import re
import json

test_ua = 'Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36'

options = Options()

# options.add_argument("--headless")  # Remove this if you want to see the browser (Headless makes the chromedriver not have a GUI)
options.add_argument("--window-size=1920,1080")

options.add_argument(f'--user-agent={test_ua}')
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")

driver = webdriver.Chrome()  # or your browser of choice


In [2]:
def ticketweb_search(s):
    if s in [None, np.nan]:
        return "lol"
    else:
        s = s.replace(" ", "+")
        return "https://www.ticketweb.ca/search?q="+s


In [3]:
path = "../../Documents/Ticket Sales.xlsx"
events = pd.read_excel(path, sheet_name ="Events")
ticketweb = pd.read_excel(path, sheet_name ="TicketWeb")    


In [4]:




# Initialize lists to store upcoming artists, ticket tiers, and ticket prices
upcoming_events = []

# Get the current date
current_date = datetime.today()

# Iterate through each row in the sales DataFrame
for index, row in events.iterrows():
    # Check if the event date is in the future
    if current_date > row["Date"]:
        pass
    # Generate the TicketWeb search URL for the artist
    ticket_url = ticketweb_search(row["Artist"])

    # Open the URL in the browser
    driver.get(ticket_url)
    script_element = driver.find_element(By.XPATH, "//script[@type='application/ld+json']")
    json_content = script_element.get_attribute('innerHTML')
    event_data = json.loads(json_content)
    # print(json.dumps(event_data, indent=4))
    if len(event_data) == 0:
        continue

    # Check if the event date in the JSON matches the date in the row
    event_date_json = datetime.strptime(event_data[0]["startDate"], "%Y-%m-%dT%H:%M").date()
    if row["Date"].date() != event_date_json:
        continue
    if row["Artist"].lower() not in event_data[0]["name"].lower():
        continue
    if row["Venue"].lower() not in event_data[0]["location"]["name"].lower():
        continue


    try:
        # Click on the first search result
        driver.find_element(By.CSS_SELECTOR, "a.is-state-default").click()
        # Find the script element containing the JSON-LD data
        

        # Check if the page source contains the word "sold out"
        if "sold out" in driver.page_source.lower():
            tier_number = "Sold Out"
            price_text = 0
        
        # Extract the ticket tier information
        else:
            tier_elements = driver.find_elements(By.CLASS_NAME, "term.theme-title")
            tier_text = driver.find_element(By.CSS_SELECTOR, "#edp-section-tickets").text
            
            # Extract the first number in the string
            if re.search(r'\d+', tier_elements[0].text):
                tier_number = re.search(r'\d+', tier_elements[0].text).group()
            elif re.search(r'\d+', tier_elements[1].text):
                tier_number = re.search(r'\d+', tier_elements[1].text).group()
            else:
                tier_number = 1
            price_text = driver.find_element(By.CSS_SELECTOR, "span.price").text
        
        # Append the extracted information to the list of dictionaries
        upcoming_events.append({
            "Artist": row["Artist"],
            "Date": row["Date"],
            "Venue": row["Venue"],
            "Tier": tier_number,
            "TicketWeb": price_text
        })
        
        # Print the extracted information
        print(upcoming_events[-1])
    except Exception as e:
        # print(row["Artist"], "Not Found", e)
        pass


{'Artist': 'Emorfik', 'Date': Timestamp('2024-10-31 00:00:00'), 'Venue': 'AXIS', 'Tier': '3', 'TicketWeb': '$49.00'}
{'Artist': 'Kaivon', 'Date': Timestamp('2024-11-01 00:00:00'), 'Venue': 'NOIR', 'Tier': '2', 'TicketWeb': '$33.74'}
{'Artist': 'Lavern', 'Date': Timestamp('2024-11-01 00:00:00'), 'Venue': 'DPRTMNT', 'Tier': '2', 'TicketWeb': '$34.74'}
{'Artist': 'Timmy Trumpet', 'Date': Timestamp('2024-11-01 00:00:00'), 'Venue': 'REBEL', 'Tier': '4', 'TicketWeb': '$67.48'}
{'Artist': 'Vini Vici', 'Date': Timestamp('2024-11-08 00:00:00'), 'Venue': 'DPRTMNT', 'Tier': '4', 'TicketWeb': '$50.78'}
{'Artist': 'Sullivan King', 'Date': Timestamp('2024-11-15 00:00:00'), 'Venue': 'REBEL', 'Tier': '3', 'TicketWeb': '$48.43'}
{'Artist': 'Ship Wrek', 'Date': Timestamp('2024-11-15 00:00:00'), 'Venue': 'DPRTMNT', 'Tier': '2', 'TicketWeb': '$31.74'}
{'Artist': 'Becky Hill', 'Date': Timestamp('2024-11-21 00:00:00'), 'Venue': 'Axis', 'Tier': 'Sold Out', 'TicketWeb': 0}
{'Artist': 'Virtual Riot', 'Date': T

In [5]:
# Combine the lists of upcoming artists, ticket tiers, and ticket prices into a DataFrame
new_ticketweb = pd.DataFrame(upcoming_events)

# Extract numeric values from the 'TicketWeb' column (ticket prices)
new_ticketweb['TicketWeb'] = new_ticketweb['TicketWeb'].str.extract('(\\d+\\.\\d+)')

# Remove rows from ticketweb where the artist and date match those in new_ticketweb
# Perform a left join of ticketweb with new_ticketweb on 'Artist' and 'Date'
merged_ticketweb = pd.merge(ticketweb, new_ticketweb, on=['Artist', 'Date'], how='outer', suffixes=('', '_new'))


In [6]:
merged_ticketweb

,Artist,Date,Tier,TicketWeb,Venue,Venue_new,Tier_new,TicketWeb_new
0,Alan Walker,2025-02-07,4,92.86,Rebel,REBEL,4,92.86
1,AlleyCVT,2024-11-01,Sold Out,NaN,NaN,NaN,NaN,NaN
2,Becky Hill,2024-11-21,Sold Out,NaN,NaN,Axis,Sold Out,NaN
3,Dion Timmer,2024-10-26,2,40.09,Noir,NaN,NaN,NaN
4,Emorfik,2024-10-31,3,49.00,NaN,AXIS,3,49.00
5,Kaivon,2024-11-01,2,33.74,Noir,NOIR,2,33.74
6,Lavern,2024-11-01,2,34.74,Dprt,DPRTMNT,2,34.74
7,Layz,2024-12-13,1,49.00,NaN,Axis,1,49.00
8,Lilly Palmer,2024-12-26,2,44.43,NaN,DPRTMNT,2,44.43
9,MK,2024-11-22,2,42.09,NaN,NaN,NaN,NaN


In [7]:
merged_ticketweb['Tier'] = merged_ticketweb.apply(
    lambda row: 'Sold Out' if row['Tier_new'] == 'Sold Out' else row['Tier'], axis=1)

# Update 'Tier' column with 'Tier_new' where 'Tier' is less than 'Tier_new'
# Update 'Tier' column with 'Tier_new' where 'Tier' is NaN
merged_ticketweb['Tier'] = merged_ticketweb.apply(
    lambda row: row['Tier_new'] if pd.isna(row['Tier']) else row['Tier'], axis=1
)
merged_ticketweb['Tier'] = merged_ticketweb.apply(
    lambda row: row['Tier_new'] if pd.to_numeric(row['Tier'], errors='coerce') < pd.to_numeric(row['Tier_new'], errors='coerce') else row['Tier'], axis=1
)
# Update 'TicketWeb' column with 'TicketWeb_new' where 'TicketWeb' is less than 'TicketWeb_new'
merged_ticketweb['TicketWeb'] = merged_ticketweb.apply(
    lambda row: row['TicketWeb_new'] if pd.to_numeric(row['TicketWeb'], errors='coerce') < pd.to_numeric(row['TicketWeb_new'], errors='coerce') else row['TicketWeb'], axis=1
)
# Update 'TicketWeb' column with 'TicketWeb_new' where 'TicketWeb' is NaN
merged_ticketweb['TicketWeb'] = merged_ticketweb.apply(
    lambda row: row['TicketWeb_new'] if pd.isna(row['TicketWeb']) else row['TicketWeb'], axis=1
)


merged_ticketweb

,Artist,Date,Tier,TicketWeb,Venue,Venue_new,Tier_new,TicketWeb_new
0,Alan Walker,2025-02-07,4,92.86,Rebel,REBEL,4,92.86
1,AlleyCVT,2024-11-01,Sold Out,NaN,NaN,NaN,NaN,NaN
2,Becky Hill,2024-11-21,Sold Out,NaN,NaN,Axis,Sold Out,NaN
3,Dion Timmer,2024-10-26,2,40.09,Noir,NaN,NaN,NaN
4,Emorfik,2024-10-31,3,49.0,NaN,AXIS,3,49.00
5,Kaivon,2024-11-01,2,33.74,Noir,NOIR,2,33.74
6,Lavern,2024-11-01,2,34.74,Dprt,DPRTMNT,2,34.74
7,Layz,2024-12-13,1,49.0,NaN,Axis,1,49.00
8,Lilly Palmer,2024-12-26,2,44.43,NaN,DPRTMNT,2,44.43
9,MK,2024-11-22,2,42.09,NaN,NaN,NaN,NaN


In [8]:


merged_ticketweb.drop(columns=['Tier_new', 'TicketWeb_new', 'Venue_new'], inplace=True, errors='ignore')
# # Save the DataFrame to the Excel file, replacing the existing sheet if it exists
with pd.ExcelWriter(path, mode='a', engine="openpyxl", if_sheet_exists="replace") as writer:
    merged_ticketweb.to_excel(writer, sheet_name="TicketWeb", index=False)



# Sold out Shows


In [9]:
driver.get("https://www.ticketweb.ca/search?q=Toronto")

venues = []
artist = []
dates = []
while True:
    # Find all event elements
    events = driver.find_elements(By.CSS_SELECTOR, "div.row.list-group-item.theme-separator-strokes")

    # Iterate through each event element
    for event in events:
        # Check if the event status is "Sold Out"
        status = event.find_element(By.CSS_SELECTOR, "div.event-status").text
        if status.lower() == "sold out":
            # Extract and return the event name
            main_info = event.find_element(By.CSS_SELECTOR, "a.main-info.theme-primary-color.hidden-xs").text
            date_text = event.find_element(By.CSS_SELECTOR, "div.event-date span.main-info").get_attribute("title")
            event_date = datetime.strptime(date_text, "%a %b %d")
            # date_info = event.find_element(By.CSS_SELECTOR, "div.event-date").text
            event_name = event.find_element(By.CSS_SELECTOR, "div.event-name a").text
            
            artist.append(event_name)
            dates.append(event_date)
            venues.append(main_info)

            # Click the next button to go to the next page of events
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Next']")
        next_button.click()
    except:
        break    

    # Create a DataFrame for venues and artists
venues_artists_df = pd.DataFrame({
    'Venue': venues,
    'Artist': artist
})

# Display the DataFrame
venues_artists_df
# Drop duplicates to keep only unique venue-artist pairs
venues_artists_df = venues_artists_df.drop_duplicates()


In [10]:
venues_artists_df

,Venue,Artist
0,CASA LOMA,Legends of Horror
1,NOIR (REBEL ENTERTAINMENT COMPLEX),ZACK FOX - DJ SET
3,VELVET UNDERGROUND,"Sasha Alex Sloan - ""Me Again"" Tour"
5,THE AXIS CLUB,"ALLEYCVT - ""FEEL IT - FALL TOUR 2024"""
6,CODA,AC Slater
7,THE PHOENIX CONCERT THEATRE,Zara Larsson
8,HARD LUCK BAR,Saturdays At Your Place w/ Carpool & Harrison ...
9,DRAKE UNDERGROUND,IAMDDB
10,THE AXIS CLUB,Becky Hill
11,LEE'S PALACE,BLOOD INCANTATION - ABSOLUTE ELSETOUR
